<a href="https://colab.research.google.com/github/cjakuc/DS-Unit-2-Linear-Models/blob/master/module3-ridge-regression/LS_DS_213_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 3*

---

# Ridge Regression

## Assignment

We're going back to our other **New York City** real estate dataset. Instead of predicting apartment rents, you'll predict property sales prices.

But not just for condos in Tribeca...

- [x] Use a subset of the data where `BUILDING_CLASS_CATEGORY` == `'01 ONE FAMILY DWELLINGS'` and the sale price was more than 100 thousand and less than 2 million.
- [x] Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.
- [x] Do one-hot encoding of categorical features.
- [x] Do feature selection with `SelectKBest`.
- [x] Fit a ridge regression model with multiple features. Use the `normalize=True` parameter (or do [feature scaling](https://scikit-learn.org/stable/modules/preprocessing.html) beforehand — use the scaler's `fit_transform` method with the train set, and the scaler's `transform` method with the test set)
- [x] Get mean absolute error for the test set.
- [x] As always, commit your notebook to your fork of the GitHub repo.

The [NYC Department of Finance](https://www1.nyc.gov/site/finance/taxes/property-rolling-sales-data.page) has a glossary of property sales terms and NYC Building Class Code Descriptions. The data comes from the [NYC OpenData](https://data.cityofnewyork.us/browse?q=NYC%20calendar%20sales) portal.


## Stretch Goals

Don't worry, you aren't expected to do all these stretch goals! These are just ideas to consider and choose from.

- [ ] Add your own stretch goal(s) !
- [ ] Instead of `Ridge`, try `LinearRegression`. Depending on how many features you select, your errors will probably blow up! 💥
- [ ] Instead of `Ridge`, try [`RidgeCV`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html).
- [ ] Learn more about feature selection:
    - ["Permutation importance"](https://www.kaggle.com/dansbecker/permutation-importance)
    - [scikit-learn's User Guide for Feature Selection](https://scikit-learn.org/stable/modules/feature_selection.html)
    - [mlxtend](http://rasbt.github.io/mlxtend/) library
    - scikit-learn-contrib libraries: [boruta_py](https://github.com/scikit-learn-contrib/boruta_py) & [stability-selection](https://github.com/scikit-learn-contrib/stability-selection)
    - [_Feature Engineering and Selection_](http://www.feat.engineering/) by Kuhn & Johnson.
- [ ] Try [statsmodels](https://www.statsmodels.org/stable/index.html) if you’re interested in more inferential statistical approach to linear regression and feature selection, looking at p values and 95% confidence intervals for the coefficients.
- [ ] Read [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf), Chapters 1-3, for more math & theory, but in an accessible, readable way.
- [ ] Try [scikit-learn pipelines](https://scikit-learn.org/stable/modules/compose.html).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import pandas as pd
import pandas_profiling

# Read New York City property sales data
df = pd.read_csv(DATA_PATH+'condos/NYC_Citywide_Rolling_Calendar_Sales.csv')

# Change column names: replace spaces with underscores
df.columns = [col.replace(' ', '_') for col in df]

# SALE_PRICE was read as strings.
# Remove symbols, convert to integer
df['SALE_PRICE'] = (
    df['SALE_PRICE']
    .str.replace('$','')
    .str.replace('-','')
    .str.replace(',','')
    .astype(int)
)

In [0]:
# BOROUGH is a numeric column, but arguably should be a categorical feature,
# so convert it from a number to a string
df['BOROUGH'] = df['BOROUGH'].astype(str)

In [0]:
# Reduce cardinality for NEIGHBORHOOD feature

# Get a list of the top 10 neighborhoods
top10 = df['NEIGHBORHOOD'].value_counts()[:10].index

# At locations where the neighborhood is NOT in the top 10, 
# replace the neighborhood with 'OTHER'
df.loc[~df['NEIGHBORHOOD'].isin(top10), 'NEIGHBORHOOD'] = 'OTHER'

## 1) Use a subset of the data where BUILDING_CLASS_CATEGORY == '01 ONE FAMILY DWELLINGS' and the sale price was more than 100 thousand and less than 2 million.

In [7]:
condition = ((df['BUILDING_CLASS_CATEGORY']=='01 ONE FAMILY DWELLINGS') &
             (df['SALE_PRICE'] > 100000) &
             (df['SALE_PRICE'] < 2000000))
df = df[condition]
# Check for max/min sale price
print(df.describe())
df.head()

              BLOCK          LOT  ...  TAX_CLASS_AT_TIME_OF_SALE    SALE_PRICE
count   3151.000000  3151.000000  ...                     3151.0  3.151000e+03
mean    6917.976515    75.981593  ...                        1.0  6.285601e+05
std     3963.326705   161.089514  ...                        0.0  2.929904e+05
min       21.000000     1.000000  ...                        1.0  1.040000e+05
25%     4016.000000    21.000000  ...                        1.0  4.475000e+05
50%     6301.000000    42.000000  ...                        1.0  5.680000e+05
75%    10208.500000    69.000000  ...                        1.0  7.600000e+05
max    16350.000000  2720.000000  ...                        1.0  1.955000e+06

[8 rows x 11 columns]


,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BLOCK,LOT,EASE-MENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,LAND_SQUARE_FEET,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_PRICE,SALE_DATE
44,3,OTHER,01 ONE FAMILY DWELLINGS,1,5495,801,NaN,A9,4832 BAY PARKWAY,NaN,11230.0,1.0,0.0,1.0,"6,800",1325.0,1930.0,1,A9,550000,01/01/2019
61,4,OTHER,01 ONE FAMILY DWELLINGS,1,7918,72,NaN,A1,80-23 232ND STREET,NaN,11427.0,1.0,0.0,1.0,"4,000",2001.0,1940.0,1,A1,200000,01/01/2019
78,2,OTHER,01 ONE FAMILY DWELLINGS,1,4210,19,NaN,A1,1260 RHINELANDER AVE,NaN,10461.0,1.0,0.0,1.0,"3,500",2043.0,1925.0,1,A1,810000,01/02/2019
108,3,OTHER,01 ONE FAMILY DWELLINGS,1,5212,69,NaN,A1,469 E 25TH ST,NaN,11226.0,1.0,0.0,1.0,"4,000",2680.0,1899.0,1,A1,125000,01/02/2019
111,3,OTHER,01 ONE FAMILY DWELLINGS,1,7930,121,NaN,A5,5521 WHITTY LANE,NaN,11203.0,1.0,0.0,1.0,"1,710",1872.0,1940.0,1,A5,620000,01/02/2019


In [8]:
# Check for building class cats
df['BUILDING_CLASS_CATEGORY'].describe()

count                        3151
unique                          1
top       01 ONE FAMILY DWELLINGS
freq                         3151
Name: BUILDING_CLASS_CATEGORY, dtype: object

## 2) Do train/test split. Use data from January — March 2019 to train. Use data from April 2019 to test.

In [0]:
# Check that sale date variable is in datetime
df.dtypes
# It is not

In [0]:
# Convert sale dat to datetime
df['SALE_DATE'] = pd.to_datetime(df['SALE_DATE'],
                                 infer_datetime_format=True)
# Check that it worked
df.dtypes

In [15]:
df['SALE_DATE'].describe()

count                    3151
unique                     91
top       2019-01-31 00:00:00
freq                       78
first     2019-01-01 00:00:00
last      2019-04-30 00:00:00
Name: SALE_DATE, dtype: object

In [0]:
# Create the train and test splits
cutoff = pd.to_datetime('2019-04-01')
train = df[df['SALE_DATE'] < cutoff]
test = df[df['SALE_DATE'] > cutoff]

## 3) Do one-hot encoding of categorical features.

In [17]:
# Find categorical features with low cardinality
train.describe(exclude='number').T.sort_values(by='unique')

,count,unique,top,freq,first,last
BUILDING_CLASS_CATEGORY,2507,1,01 ONE FAMILY DWELLINGS,2507,NaT,NaT
APARTMENT_NUMBER,1,1,RP.,1,NaT,NaT
TAX_CLASS_AT_PRESENT,2507,2,1,2476,NaT,NaT
BOROUGH,2507,5,4,1204,NaT,NaT
NEIGHBORHOOD,2507,7,OTHER,2360,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,A1,919,NaT,NaT
BUILDING_CLASS_AT_PRESENT,2507,13,A1,919,NaT,NaT
SALE_DATE,2507,68,2019-01-31 00:00:00,78,2019-01-01,2019-03-30
LAND_SQUARE_FEET,2507,887,"4,000",234,NaT,NaT
ADDRESS,2507,2497,125-27 LUCAS STREET,2,NaT,NaT


In [21]:
# Find categorical features with high cardinality
train.describe(exclude='number').T.sort_values(by='unique',ascending=False)

,count,unique,top,freq,first,last
ADDRESS,2507,2497,125-27 LUCAS STREET,2,NaT,NaT
LAND_SQUARE_FEET,2507,887,"4,000",234,NaT,NaT
SALE_DATE,2507,68,2019-01-31 00:00:00,78,2019-01-01,2019-03-30
BUILDING_CLASS_AT_PRESENT,2507,13,A1,919,NaT,NaT
BUILDING_CLASS_AT_TIME_OF_SALE,2507,11,A1,919,NaT,NaT
NEIGHBORHOOD,2507,7,OTHER,2360,NaT,NaT
BOROUGH,2507,5,4,1204,NaT,NaT
TAX_CLASS_AT_PRESENT,2507,2,1,2476,NaT,NaT
BUILDING_CLASS_CATEGORY,2507,1,01 ONE FAMILY DWELLINGS,2507,NaT,NaT
APARTMENT_NUMBER,1,1,RP.,1,NaT,NaT


In [19]:
train.describe()

,BLOCK,LOT,EASE-MENT,ZIP_CODE,RESIDENTIAL_UNITS,COMMERCIAL_UNITS,TOTAL_UNITS,GROSS_SQUARE_FEET,YEAR_BUILT,TAX_CLASS_AT_TIME_OF_SALE,SALE_PRICE
count,2507.000000,2507.000000,0.0,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.000000,2507.0,2.507000e+03
mean,6758.303949,75.778221,NaN,10993.398484,0.987635,0.016354,1.003989,1473.744715,1944.766653,1.0,6.215737e+05
std,3975.909029,157.531138,NaN,494.291462,0.110532,0.129966,0.171794,599.217635,27.059337,0.0,2.916072e+05
min,21.000000,1.000000,NaN,10301.000000,0.000000,0.000000,0.000000,0.000000,1890.000000,1.0,1.040000e+05
25%,3837.500000,21.000000,NaN,10314.000000,1.000000,0.000000,1.000000,1144.000000,1925.000000,1.0,4.405000e+05
50%,6022.000000,42.000000,NaN,11234.000000,1.000000,0.000000,1.000000,1368.000000,1940.000000,1.0,5.600000e+05
75%,9888.500000,70.000000,NaN,11413.000000,1.000000,0.000000,1.000000,1683.000000,1960.000000,1.0,7.500000e+05
max,16350.000000,2720.000000,NaN,11697.000000,1.000000,2.000000,3.000000,7875.000000,2018.000000,1.0,1.955000e+06


In [20]:
train.describe(exclude='number')

,BOROUGH,NEIGHBORHOOD,BUILDING_CLASS_CATEGORY,TAX_CLASS_AT_PRESENT,BUILDING_CLASS_AT_PRESENT,ADDRESS,APARTMENT_NUMBER,LAND_SQUARE_FEET,BUILDING_CLASS_AT_TIME_OF_SALE,SALE_DATE
count,2507,2507,2507,2507,2507,2507,1,2507,2507,2507
unique,5,7,1,2,13,2497,1,887,11,68
top,4,OTHER,01 ONE FAMILY DWELLINGS,1,A1,125-27 LUCAS STREET,RP.,"4,000",A1,2019-01-31 00:00:00
freq,1204,2360,2507,2476,919,2,1,234,919,78
first,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-01-01 00:00:00
last,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2019-03-30 00:00:00


In [0]:
# Exclude the features with high cardinality & NaNs
target = 'SALE_PRICE'
exclude = ['ADDRESS',
           'LAND_SQUARE_FEET',
           'SALE_DATE',
           'APARTMENT_NUMBER',
           'EASE-MENT',
           'BUILDING_CLASS_CATEGORY']
features = train.columns.drop([target] + exclude)

In [0]:
# Create feature splits
X_train = train[features]
y_train = train[target]
X_test = test[features]
y_test = test[target]

In [0]:
# Encode categorical features using category encoders
import category_encoders as ce
encoder = ce.OneHotEncoder(use_cat_names=True)
X_train = encoder.fit_transform(X_train)

In [0]:
X_test = encoder.transform(X_test)

In [26]:
X_train.columns

Index(['BOROUGH_3', 'BOROUGH_4', 'BOROUGH_2', 'BOROUGH_5', 'BOROUGH_1',
       'NEIGHBORHOOD_OTHER', 'NEIGHBORHOOD_FLUSHING-NORTH',
       'NEIGHBORHOOD_EAST NEW YORK', 'NEIGHBORHOOD_BEDFORD STUYVESANT',
       'NEIGHBORHOOD_FOREST HILLS', 'NEIGHBORHOOD_BOROUGH PARK',
       'NEIGHBORHOOD_ASTORIA', 'TAX_CLASS_AT_PRESENT_1',
       'TAX_CLASS_AT_PRESENT_1D', 'BLOCK', 'LOT',
       'BUILDING_CLASS_AT_PRESENT_A9', 'BUILDING_CLASS_AT_PRESENT_A1',
       'BUILDING_CLASS_AT_PRESENT_A5', 'BUILDING_CLASS_AT_PRESENT_A0',
       'BUILDING_CLASS_AT_PRESENT_A2', 'BUILDING_CLASS_AT_PRESENT_A3',
       'BUILDING_CLASS_AT_PRESENT_S1', 'BUILDING_CLASS_AT_PRESENT_A4',
       'BUILDING_CLASS_AT_PRESENT_A6', 'BUILDING_CLASS_AT_PRESENT_A8',
       'BUILDING_CLASS_AT_PRESENT_B2', 'BUILDING_CLASS_AT_PRESENT_S0',
       'BUILDING_CLASS_AT_PRESENT_B3', 'ZIP_CODE', 'RESIDENTIAL_UNITS',
       'COMMERCIAL_UNITS', 'TOTAL_UNITS', 'GROSS_SQUARE_FEET', 'YEAR_BUILT',
       'TAX_CLASS_AT_TIME_OF_SALE', 'BUILDING_CLA

## 4) Do feature selection with `SelectKBest`.

In [28]:
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.metrics import mean_absolute_error

for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')
    
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    model = LinearRegression()
    model.fit(X_train_selected, y_train)
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test, y_pred)
    print(f'Test Mean Absolute Error: ${mae:,.0f} \n')

1 features
Test Mean Absolute Error: $184,772 

2 features
Test Mean Absolute Error: $183,439 

3 features
Test Mean Absolute Error: $183,439 

4 features
Test Mean Absolute Error: $186,145 

5 features
Test Mean Absolute Error: $184,511 

6 features
Test Mean Absolute Error: $178,882 

7 features
Test Mean Absolute Error: $179,387 

8 features
Test Mean Absolute Error: $179,614 

9 features
Test Mean Absolute Error: $175,102 

10 features
Test Mean Absolute Error: $174,561 

11 features
Test Mean Absolute Error: $174,202 

12 features
Test Mean Absolute Error: $165,070 

13 features
Test Mean Absolute Error: $165,941 

14 features
Test Mean Absolute Error: $164,898 

15 features
Test Mean Absolute Error: $164,898 

16 features
Test Mean Absolute Error: $164,748 

17 features
Test Mean Absolute Error: $164,748 

18 features
Test Mean Absolute Error: $164,487 

19 features
Test Mean Absolute Error: $164,487 

20 features
Test Mean Absolute Error: $159,889 

21 features
Test Mean Absolut

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered


22 features
Test Mean Absolute Error: $159,988 

23 features
Test Mean Absolute Error: $159,988 

24 features
Test Mean Absolute Error: $159,988 

25 features
Test Mean Absolute Error: $159,988 

26 features
Test Mean Absolute Error: $159,988 

27 features
Test Mean Absolute Error: $159,988 

28 features
Test Mean Absolute Error: $159,988 

29 features
Test Mean Absolute Error: $160,908 

30 features
Test Mean Absolute Error: $160,998 

31 features
Test Mean Absolute Error: $160,998 

32 features
Test Mean Absolute Error: $160,998 

33 features
Test Mean Absolute Error: $160,739 

34 features
Test Mean Absolute Error: $160,739 

35 features
Test Mean Absolute Error: $160,732 

36 features
Test Mean Absolute Error: $160,390 

37 features
Test Mean Absolute Error: $160,390 

38 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

Test Mean Absolute Error: $161,297 

39 features
Test Mean Absolute Error: $160,602 

40 features
Test Mean Absolute Error: $160,645 

41 features
Test Mean Absolute Error: $160,669 

42 features
Test Mean Absolute Error: $160,669 

43 features
Test Mean Absolute Error: $160,669 

44 features
Test Mean Absolute Error: $160,669 

45 features
Test Mean Absolute Error: $160,670 

46 features
Test Mean Absolute Error: $160,668 

47 features
Test Mean Absolute Error: $160,669 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

## 5) Fit a ridge regression model with multiple features. Use the normalize=True parameter (or do feature scaling beforehand — use the scaler's fit_transform method with the train set, and the scaler's transform method with the test set)

In [37]:
from sklearn.linear_model import RidgeCV

alphas = [0.01, 0.1, 1.0, 10.0, 100.0]
for k in range(1, len(X_train.columns)+1):
    print(f'{k} features')
    
    # Select the k best features
    selector = SelectKBest(score_func=f_regression, k=k)
    X_train_selected = selector.fit_transform(X_train, y_train)
    X_test_selected = selector.transform(X_test)

    # Select the best alpha for ridge regression using RidgeCV
    model = RidgeCV(alphas=alphas,
                    normalize=True)
    model.fit(X_train_selected,
              y_train)
    
    # Find the MAE of the model w/ selected alpha and selected k features
    y_pred = model.predict(X_test_selected)
    mae = mean_absolute_error(y_test,
                              y_pred)
    print(f'Alpha: {model.alpha_} and Test Mean Absolute Error: ${mae:,.0f} \n')

1 features
Alpha: 0.01 and Test Mean Absolute Error: $184,831 

2 features
Alpha: 0.01 and Test Mean Absolute Error: $183,511 

3 features
Alpha: 0.01 and Test Mean Absolute Error: $183,505 

4 features
Alpha: 0.01 and Test Mean Absolute Error: $186,135 

5 features
Alpha: 0.01 and Test Mean Absolute Error: $184,473 

6 features
Alpha: 0.01 and Test Mean Absolute Error: $178,895 

7 features
Alpha: 0.01 and Test Mean Absolute Error: $179,294 

8 features
Alpha: 0.01 and Test Mean Absolute Error: $179,495 

9 features
Alpha: 0.01 and Test Mean Absolute Error: $176,273 

10 features
Alpha: 0.01 and Test Mean Absolute Error: $175,572 

11 features
Alpha: 0.01 and Test Mean Absolute Error: $175,402 

12 features
Alpha: 0.01 and Test Mean Absolute Error: $165,211 

13 features
Alpha: 0.01 and Test Mean Absolute Error: $165,616 

14 features
Alpha: 0.01 and Test Mean Absolute Error: $164,400 

15 features
Alpha: 0.01 and Test Mean Absolute Error: $164,402 

16 features
Alpha: 0.01 and Test M

/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

Alpha: 0.01 and Test Mean Absolute Error: $163,986 

19 features
Alpha: 0.01 and Test Mean Absolute Error: $163,985 

20 features
Alpha: 0.01 and Test Mean Absolute Error: $159,476 

21 features
Alpha: 0.01 and Test Mean Absolute Error: $159,365 

22 features
Alpha: 0.01 and Test Mean Absolute Error: $159,384 

23 features
Alpha: 0.01 and Test Mean Absolute Error: $159,384 

24 features
Alpha: 0.01 and Test Mean Absolute Error: $159,416 

25 features
Alpha: 0.01 and Test Mean Absolute Error: $159,421 

26 features
Alpha: 0.01 and Test Mean Absolute Error: $159,423 

27 features
Alpha: 0.01 and Test Mean Absolute Error: $159,424 

28 features
Alpha: 0.01 and Test Mean Absolute Error: $159,425 

29 features
Alpha: 0.01 and Test Mean Absolute Error: $160,366 

30 features
Alpha: 0.01 and Test Mean Absolute Error: $160,409 

31 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

Alpha: 0.01 and Test Mean Absolute Error: $160,410 

32 features
Alpha: 0.01 and Test Mean Absolute Error: $160,421 

33 features
Alpha: 0.01 and Test Mean Absolute Error: $160,140 

34 features
Alpha: 0.01 and Test Mean Absolute Error: $160,141 

35 features
Alpha: 0.01 and Test Mean Absolute Error: $160,132 

36 features
Alpha: 0.01 and Test Mean Absolute Error: $159,667 

37 features
Alpha: 0.01 and Test Mean Absolute Error: $159,666 

38 features
Alpha: 0.01 and Test Mean Absolute Error: $160,359 

39 features
Alpha: 0.01 and Test Mean Absolute Error: $159,741 

40 features
Alpha: 0.01 and Test Mean Absolute Error: $159,752 

41 features
Alpha: 0.01 and Test Mean Absolute Error: $159,805 

42 features


/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered

Alpha: 0.01 and Test Mean Absolute Error: $159,798 

43 features
Alpha: 0.01 and Test Mean Absolute Error: $159,792 

44 features
Alpha: 0.01 and Test Mean Absolute Error: $159,791 

45 features
Alpha: 0.01 and Test Mean Absolute Error: $159,791 

46 features
Alpha: 0.01 and Test Mean Absolute Error: $160,066 

47 features
Alpha: 0.01 and Test Mean Absolute Error: $160,066 



/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered in true_divide
  corr /= X_norms
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:304: RuntimeWarning: invalid value encountered in true_divide
  F = corr ** 2 / (1 - corr ** 2) * degrees_of_freedom
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in greater
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:903: RuntimeWarning: invalid value encountered in less
  return (a < x) & (x < b)
/usr/local/lib/python3.6/dist-packages/scipy/stats/_distn_infrastructure.py:1912: RuntimeWarning: invalid value encountered in less_equal
  cond2 = cond0 & (x <= _a)
/usr/local/lib/python3.6/dist-packages/sklearn/feature_selection/_univariate_selection.py:299: RuntimeWarning: divide by zero encountered